04 ストップワード
==============

* 役に立たない単語を取り除くもう1つの方法として、あまりに頻出するため役に立たない単語を捨てる方法がある

* これには2つの手法がある

    * 1つは、言語固有のストップワードリストを作っておく方法
    
    * もう1つは、頻度の高い単語を捨てる方法

In [2]:
from sklearn.datasets import load_files

reviews_train = load_files("/Users/MacUser/data/aclImdb/train/")
text_train, y_train = reviews_train.data, reviews_train.target
text_train = [doc.replace(b"<br />", b" ") for doc in text_train]
reviews_test = load_files("/Users/MacUser/data/aclImdb/test/")
text_test, y_test = reviews_test.data, reviews_test.target
text_test = [doc.replace(b"<br />", b" ") for doc in text_test]

from sklearn.feature_extraction.text import CountVectorizer
vect = CountVectorizer().fit(text_train)
X_train = vect.transform(text_train)

import numpy as np
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
scores = cross_val_score(LogisticRegression(), X_train, y_train, cv=5)

from sklearn.model_selection import GridSearchCV
param_grid = {'C': [0.001, 0.01, 0.1, 1, 10]}
grid = GridSearchCV(LogisticRegression(), param_grid, cv=5)
grid.fit(X_train, y_train)

X_test = vect.transform(text_test)

vect = CountVectorizer(min_df=5).fit(text_train)
X_train = vect.transform(text_train)
grid = GridSearchCV(LogisticRegression(), param_grid, cv=5)
grid.fit(X_train, y_train)

/Users/MacUser/anaconda2/envs/tf140/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/MacUser/anaconda2/envs/tf140/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/MacUser/anaconda2/envs/tf140/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'C': [0.001, 0.01, 0.1, 1, 10]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

* scikit-learnは、英語のストップワードリストを`feature_extraction.text`モジュールに用意している

In [1]:
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS
print("Number of stop words: {}".format(len(ENGLISH_STOP_WORDS)))
print("Every 10th stopword:\n{}".format(list(ENGLISH_STOP_WORDS)[::10]))

Number of stop words: 318
Every 10th stopword:
['empty', 'always', 'somehow', 'others', 'further', 'has', 'almost', 'hundred', 'eg', 'too', 'nine', 'which', 'some', 'us', 'hereupon', 'yours', 'show', 'whither', 'interest', 'part', 'that', 'herein', 'any', 'fifteen', 'nevertheless', 'one', 'co', 'anything', 'four', 'latterly', 'whereafter', 'well']


* 当然だが、リスト中のストップワードを取り除いても特徴量の数がリストの長さ(318)だけ減ることになるが、性能は向上する可能性がある

In [3]:
# stop_words="english"を指定すると、組み込みのストップワードリストを用いる
# ストップワードリストに単語を追加したり、独自のものを使うこともできる
vect = CountVectorizer(min_df=5, stop_words="english").fit(text_train)
X_train = vect.transform(text_train)
print("X_train with stop words:\n{}".format(repr(X_train)))

X_train with stop words:
<25000x26966 sparse matrix of type '<class 'numpy.int64'>'
	with 2149958 stored elements in Compressed Sparse Row format>


* データセット中の特徴量の数は305(27,271-26,966)減っている

    * つまり、ストップワード中の全てでは無いがほとんどがデータセット中に現れた事になる
    
    * グリッドサーチを行ってみる

In [4]:
grid = GridSearchCV(LogisticRegression(), param_grid, cv=5)
grid.fit(X_train, y_train)
print("Best cross-validation score: {:.2f}".format(grid.best_score_))

/Users/MacUser/anaconda2/envs/tf140/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Best cross-validation score: 0.88


* グリッドサーチの性能は、ストップワードを使うことで少し低下している

    * 27,000から305の特徴量を除くだけでは、わざわざストップリストを使う意味はない
    
    * ただし、固定したリストは、小さいデータセットに対しては有効である
    
    * データセットそのものに、どの単語をストップワードにするべきかを決めるだけの情報が含まれていない
    


| 版   | 年/月/日   |
| ---- | ---------- |
| 初版 | 2019/04/06 |